In [2]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path
import tensorflow as tf

In [ ]:
# shamelessly taken from https://danijar.com/variable-sequence-lengths-in-tensorflow/
def getSequenceLengths(batchOfSequences):
  used = tf.sign(tf.reduce_max(tf.abs(batchOfSequences), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [ ]:
flags = tf.flags
logging = tf.logging

In [ ]:
flags.DEFINE_bool("train", True, "train mode (test mode if not)")
FLAGS = flags.FLAGS

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
df = pd.read_pickle('final_processed_invoker_data')

In [12]:
columnDict = {}
for i, v in enumerate(df.columns.values):
    columnDict[i] = v
indexDict = {v: k for k, v in columnDict.items()}
numberEntries = len(df)
batchSize = int(numberEntries/20)

In [ ]:
# 265 is highest ID, and we need a zero for empty
DIM = 266
matchItemsTensor = tf.placeholder(tf.float32, shape=shape=[None, None, DIM])
batched_data = tf.train.batch(
    tensors=[matchItemsTensor],
    batch_size=batchSize,
    dynamic_pad=True,
    name="lstm_batch"
)

# shift each sequence to the left by one (stride to the right by 1)
# and then pad at the end to maintain same lengths!
batched_data_strided = tf.strided_slice(batched_data, [0, 1, 0], batched_data.shape)
target = tf.pad(batched_data_strided, [[0, 0], [0, 1], [0, 0]], "CONSTANT")

batchSequenceLengths = getSequenceLengths(batched_data)
lstmHiddenSize = 50
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmHiddenSize)

In [ ]:
lstmOutput, last_states = tf.nn.dynamic_rnn(
    cell=lstmCell,
    dtype=tf.float64,
    sequence_length=batchSequenceLengths,
    inputs=batched_data)

In [ ]:
lstmOutput = tf.reshape(lstmOutput, [-1, lstmHiddenSize])

In [ ]:
comboLayer = tf.concat([lstmOutput, heroInfoMap], 1)
prediction = tf.nn.softmax(tf.matmul(output, weight) + bias)
logics = tf.

In [ ]:
variable_dict = {'weights_1': weights_1, 'weights_2': weights_2,
                     'bias_1': bias_1, 'bias_2': bias_2}
saver = tf.train.Saver(variable_dict)
init = tf.global_variables_initializer()

In [6]:
c = tf.constant([[
            [0, 0, 0], [1, 1, 1], [2, 2, 2]],[
             [3, 3, 3], [4,4,4], [5,5,5]    ]])

In [33]:
c_strided = tf.strided_slice(c, [0, 1, 0], c.shape)

c_padded = tf.pad(c_strided, [[0, 0], [0, 1], [0, 0]], "CONSTANT")

In [34]:
print(c.shape)

(2, 3, 3)


In [35]:
with tf.Session() as sess:
    c_val = sess.run(c)
    c_strided_val = sess.run(c_strided)
    c_padded_val = sess.run(c_padded)
    print(c_val)
    print(c_val[0])
    print(c_val[1])
    print("stride")
    print(c_strided_val)
    print(c_strided_val[0])
    print(c_strided_val[1])
    print("pad")
    print(c_padded_val)
    print(c_padded_val[0])
    print(c_padded_val[1])

[[[0 0 0]
  [1 1 1]
  [2 2 2]]

 [[3 3 3]
  [4 4 4]
  [5 5 5]]]
[[0 0 0]
 [1 1 1]
 [2 2 2]]
[[3 3 3]
 [4 4 4]
 [5 5 5]]
stride
[[[1 1 1]
  [2 2 2]]

 [[4 4 4]
  [5 5 5]]]
[[1 1 1]
 [2 2 2]]
[[4 4 4]
 [5 5 5]]
pad
[[[1 1 1]
  [2 2 2]
  [0 0 0]]

 [[4 4 4]
  [5 5 5]
  [0 0 0]]]
[[1 1 1]
 [2 2 2]
 [0 0 0]]
[[4 4 4]
 [5 5 5]
 [0 0 0]]
